<a href="https://colab.research.google.com/github/TamizharasanG/PhonePePulse/blob/main/PhonePe_Pulse_Dashboard1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pathlib import Path

! git clone "https://github.com/PhonePe/pulse"

In [15]:
%%writefile etl.py
import pandas as pd
from pathlib import Path

#!git clone "https://github.com/PhonePe/pulse"

input_file = Path.cwd() / "pulse"
file = list(input_file.rglob("*.json*"))

end_val = []
for i in file:
  full_path = i.absolute()
  my_path = full_path.as_posix()
  val = my_path.split("/")
  num = 0
  for j in val:
    if j == "aggregated":
      num+= 1
    elif j == "map":
      num+=100
    elif j == "top":
      num+= 200
    elif j == "state":
      num +=50
    elif j == "transaction":
      num+= 10
    elif j == "user":
      num+= 25
  new_val = str(num)+"!"+my_path
  end_val.append(new_val)

agg_trans_year_list = []
agg_trans_state_list = []
agg_user_year_list = []
agg_user_state_list = []
map_trans_year_list = []
map_trans_state_list = []
map_user_year_list = []
map_user_state_list = []
top_trans_year_list = []
top_trans_state_list = []
top_user_year_list = []
top_user_state_list = []
for i in end_val:
  tr = i.split("!")
  if int(tr[0]) == 11:
    agg_trans_year_list.append(tr[1])
  elif int(tr[0]) == 61:
    agg_trans_state_list.append(tr[1])
  elif int(tr[0]) == 26:
    agg_user_year_list.append(tr[1])
  elif int(tr[0]) == 76:
    agg_user_state_list.append(tr[1])
  elif int(tr[0]) == 110:
    map_trans_year_list.append(tr[1])
  elif int(tr[0]) == 160:
    map_trans_state_list.append(tr[1])
  elif int(tr[0]) == 125:
    map_user_year_list.append(tr[1])
  elif int(tr[0]) == 175:
    map_user_state_list.append(tr[1])
  elif int(tr[0]) == 210:
    top_trans_year_list.append(tr[1])
  elif int(tr[0]) == 260:
    top_trans_state_list.append(tr[1])
  elif int(tr[0]) == 225:
    top_user_year_list.append(tr[1])
  elif int(tr[0]) == 275:
    top_user_state_list.append(tr[1])

#logic 1 : Agg_trans_year
def agg_trans_year(x):
  df_raw = pd.read_json(x)
  category = []
  count = []
  amount = []
  year = []
  quadrant = []
  for i in df_raw.iloc[2, 2]:
    category.append(i["name"])
    count.append(i["paymentInstruments"][0]["count"])
    amount.append(i["paymentInstruments"][0]["amount"])
    year.append(int(x.split("/")[-2]))
    quadrant.append(int(x.split("/")[-1].split(".")[0]))
  df = pd.DataFrame({"Year" : year,
                     "Quadrant" : quadrant,
                     "Category" : category,
                     "Total Count" : count,
                     "Total Amount" : amount,
                          })

  return df

#function for combining data from all links
def tabulate_agg_trans_year(x):
  df1 = agg_trans_year(x[0])
  for i in x[1:]:
    df1 = df1.append(agg_trans_year(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 2 : Agg_trans_state
def agg_trans_state(x):
  df_raw = pd.read_json(x)
  name = x.split("/")[-3].replace("&", " ").replace("-", " ").title()
  name = " ".join(name.split())
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1].split(".")[0])

  info = {"State Name" : name, "Year" : year, "Quadrant" : quadrant}
  for i in df_raw.iloc[2, 2]:
    info[i["name"]+" "+"Count"] = i["paymentInstruments"][0]["count"]
    info[i["name"]+" "+"Value"] = i["paymentInstruments"][0]["amount"]

  df = pd.DataFrame(info, index = [0])
  return df

#function for combining data from all links
def tabulate_agg_trans_state(x):
  df1 = agg_trans_state(x[0])
  for i in x[1:]:
    df1 = df1.append(agg_trans_state(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 3 : Agg_user_year
def agg_user_year(x):
  df_raw = pd.read_json(x)
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1].split(".")[0])
  info = {"Year" : year, "Quadrant" : quadrant}
  info["Registered Users"] = df_raw.iloc[0, 2]["registeredUsers"]
  info["App Opens"] = df_raw.iloc[0, 2]["appOpens"]
  if df_raw.iloc[1, 2] !=  None:
    for i in df_raw.iloc[1, 2]:
      info[i["brand"]+" "+"Users"] = i["count"]

  df = pd.DataFrame(info, index = [0])
  return df

#function for combining all data from the links
def tabulate_agg_user_year(x):
  df1 = agg_user_year(x[0])
  for i in x[1:]:
    df1 = df1.append(agg_user_year(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 4 : Agg_user_state
def agg_user_state(x):
  df_raw = pd.read_json(x)
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1].split(".")[0])
  name = x.split("/")[-3].replace("&", " ").replace("-", " ").replace(" ", " ").title()
  name = " ".join(name.split())
  info = {"State Name" : name, "Year" : year, "Quadrant" : quadrant}
  info["Registered Users"] = df_raw.iloc[0, 2]["registeredUsers"]
  info["App Opens"] = df_raw.iloc[0, 2]["appOpens"]
  if df_raw.iloc[1, 2] !=  None:
    for i in df_raw.iloc[1, 2]:
      info[i["brand"]+" "+"Users"] = i["count"]

  df = pd.DataFrame(info, index = [0])
  return df

#function for combining all data from the links
def tabulate_agg_user_state(x):
  df1 = agg_user_state(x[0])
  for i in x[1:]:
    df1 = df1.append(agg_user_state(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 5: Map_trans_state
def map_trans_state(x):
  df_raw = pd.read_json(x)
  st_name = x.split("/")[-3].replace("&", " ").replace("-", " ").replace(" ", " ").title()
  st_name = " ".join(st_name.split())
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1][0])
  state_name = []
  d_name = []
  transaction_count = []
  transaction_amount = []
  for i in df_raw.iloc[0, 2]:
    state_name.append(st_name)
    d_name.append(i["name"].title())
    transaction_count.append(i["metric"][0]["count"])
    transaction_amount.append(i["metric"][0]["amount"])
  
  info = {"State Name" : state_name,
          "District Name" : d_name,
          "Transaction Count": transaction_count,
          "Transaction Amount" : transaction_amount}

  df = pd.DataFrame(info)
  y = [year for i in range(df.shape[0])]
  q = [quadrant for i in range(df.shape[0])]
  df.insert(0, "Year", y)
  df.insert(1, "Quadrant", q)

  return df

#function for combining all data from the links
def tabulate_map_trans_state(x):
  df1 = map_trans_state(x[0])
  for i in x[1:]:
    df1 = df1.append(map_trans_state(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 6: Map_user_state
def map_user_state(x):
  df_raw = pd.read_json(x)
  st_name = x.split("/")[-3].replace("&", " ").replace("-", " ").replace(" ", " ").title()
  st_name = " ".join(st_name.split())
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1][0])
  state_name = []
  d_name = []
  registered_users = []
  app_opens = []
  for i in df_raw.iloc[0, 2]:
    state_name.append(st_name)
    d_name.append(i.title())
    registered_users.append(df_raw.iloc[0, 2][i]["registeredUsers"])
    app_opens.append((df_raw.iloc[0, 2][i]["appOpens"]))
  
  info = {"State Name" : state_name,
          "District Name" : d_name,
          "Registered Users": registered_users,
          "App Opens" : app_opens}
  
  df = pd.DataFrame(info)
  y = [year for i in range(df.shape[0])]
  q = [quadrant for i in range(df.shape[0])]
  df.insert(0, "Year", y)
  df.insert(1, "Quadrant", q)
  return df


#function for combining all data from the links
def tabulate_map_user_state(x):
  df1 = map_user_state(x[0])
  for i in x[1:]:
    df1 = df1.append(map_user_state(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 7: Top_trans_year
def top_trans_year(x):
  df_raw = pd.read_json(x)
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1][0])
  city_name = []
  city_trans_count = []
  city_trans_amount = []
  pin_trans = []
  pin_trans_count = []
  pin_trans_amount = []
  state_name = []
  state_trans_count = []
  state_trans_amount = []
  y = []
  q = []
  for i in df_raw.iloc[0, 2]:
    y.append(year)
    q.append(quadrant)
    city_name.append(i["entityName"].title())
    city_trans_count.append(i["metric"]["count"])
    city_trans_amount.append(i["metric"]["amount"])
  for i in df_raw.iloc[1, 2]:
    pin_trans.append(i["entityName"].title())
    pin_trans_count.append(i["metric"]["count"])
    pin_trans_amount.append(i["metric"]["amount"])
  for i in df_raw.iloc[2, 2]:
    state_name.append(i["entityName"].title())
    state_trans_count.append(i["metric"]["count"])
    state_trans_amount.append(i["metric"]["amount"])
  
  info = {"Year" : y,
          "Quadrant" : q,
          "Top City Name" : city_name,
          "City Transaction Count" : city_trans_count,
          "City Transaction Amount" : city_trans_amount,
          "Top Pincode" : pin_trans,
          "Pincode Transaction Count" : pin_trans_count,
          "Pincode Transaction Amount" : pin_trans_amount,
          "Top State Name" : state_name,
          "State Transaction Count" : state_trans_count,
          "State Transaction Amount" : state_trans_amount}

  df = pd.DataFrame(info)
  return df


#function for combining all data from the links
def tabulate_top_trans_year(x):
  df1 = top_trans_year(x[0])
  for i in x[1:]:
    df1 = df1.append(top_trans_year(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 8: Top_trans_state
def top_trans_state(x):
  df_raw = pd.read_json(x)
  st_name = x.split("/")[-3].replace("&", " ").replace("-", " ").replace(" ", " ").title()
  st_name = " ".join(st_name.split())
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1][0])
  district_name = []
  district_trans_count = []
  district_trans_amount = []
  pin_trans = []
  pin_trans_count = []
  pin_trans_amount = []
  for i in df_raw.iloc[0, 2]:
    district_name.append(i["entityName"].title())
    district_trans_count.append(i["metric"]["count"])
    district_trans_amount.append(i["metric"]["amount"])
  for i in df_raw.iloc[1, 2]:
    pin_trans.append(i["entityName"])
    pin_trans_count.append(i["metric"]["count"])
    pin_trans_amount.append(i["metric"]["amount"])

  from numpy import NaN
  if len(district_name) == len(pin_trans):
    pass
  elif len(district_name) != len(pin_trans):
    z = abs(len(district_name) - len(pin_trans))
    if len(district_name) < len(pin_trans):
      adjust = [NaN for i in range(z)]
      district_name.extend(adjust)
      district_trans_count.extend(adjust)
      district_trans_amount.extend(adjust)

  info = {"Top District Name" : district_name,
          "District Transaction Count" : district_trans_count,
          "District Transaction Amount" : district_trans_amount,
          "Top Pincode" : pin_trans,
          "Pincode Transaction Count" : pin_trans_count,
          "Pincode Transaction Amount" : pin_trans_amount}
  df = pd.DataFrame(info)
  y = [year for i in range(df.shape[0])]
  q = [quadrant for i in range(df.shape[0])]
  s = [st_name for i in range(df.shape[0])]
  df.insert(0, "State Name" , s)
  df.insert(1, "Year", y)
  df.insert(2, "Quadrant", q)
  return df


#function for combining all data from the links
def tabulate_top_trans_state(x):
  df1 = top_trans_state(x[0])
  for i in x[1:]:
    df1 = df1.append(top_trans_state(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 9 : Top_user_year
def top_user_year(x):
  df_raw = pd.read_json(x)
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1][0])
  district_name = []
  district_reg_users = []
  pin = []
  pin_reg_users = []
  state_name = []
  state_reg_users = []
  for i in df_raw.iloc[0, 2]:
    district_name.append(i["name"].title())
    district_reg_users.append(i["registeredUsers"])
  for i in df_raw.iloc[1, 2]:
    pin.append(i["name"])
    pin_reg_users.append(i["registeredUsers"])
  for i in df_raw.iloc[2, 2]:
    state_name.append(i["name"].title())
    state_reg_users.append(i["registeredUsers"])

  info = {"Top District Name" : district_name,
          "District Registered Users Count" : district_reg_users,
          "Top Pincode" : pin,
          "Pincode Registered Users Count" : pin_reg_users}

  df = pd.DataFrame(info)
  y = [year for i in range(1, df.shape[0]+1)]
  q = [quadrant for i in range(1, df.shape[0]+1)]
  df.insert(0, "Year", y)
  df.insert(1, "Quadrant" ,q)
  return df
  
#function for combining all data from the links
def tabulate_top_user_year(x):
  df1 = top_user_year(x[0])
  for i in x[1:]:
    df1 = df1.append(top_user_year(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

#logic 10 : Top_user_state
def top_user_state(x):
  df_raw = pd.read_json(x)
  st_name = x.split("/")[-3].replace("&", " ").replace("-", " ").replace(" ", " ").title()
  st_name = " ".join(st_name.split())
  year = int(x.split("/")[-2])
  quadrant = int(x.split("/")[-1][0])
  district_name = []
  district_reg_users = []
  pin = []
  pin_reg_users = []
  for i in df_raw.iloc[0, 2]:
    district_name.append(i["name"].title())
    district_reg_users.append(i["registeredUsers"])
  for i in df_raw.iloc[1, 2]:
    pin.append(i["name"])
    pin_reg_users.append(i["registeredUsers"])

  from numpy import NaN
  if len(district_name) == len(pin):
    pass
  elif len(district_name) != len(pin):
    z = abs(len(district_name) - len(pin))
    if len(district_name) < len(pin):
      adjust = [NaN for i in range(z)]
      district_name.extend(adjust)
      district_reg_users.extend(adjust)

  info = {"Top District Name" : district_name,
          "District Registered users Count" : district_reg_users,
          "Top Pincode" : pin,
          "Pincode Registered Users Count" : pin_reg_users}
  df = pd.DataFrame(info)
  y = [year for i in range(df.shape[0])]
  q = [quadrant for i in range(df.shape[0])]
  s = [st_name for i in range(df.shape[0])]
  df.insert(0, "State Name" , s)
  df.insert(1, "Year", y)
  df.insert(2, "Quadrant", q)
  return df

#function for combining all data from the links
def tabulate_top_user_state(x):
  df1 = top_user_state(x[0])
  for i in x[1:]:
    df1 = df1.append(top_user_state(i))
  df1 = df1.sort_values(["Year", "Quadrant"], ascending = True, ignore_index = True)
  return df1

Aggrigation_trans_year = tabulate_agg_trans_year(agg_trans_year_list)
Aggrigation_trans_state = tabulate_agg_trans_state(agg_trans_state_list)
Aggrigation_user_year = tabulate_agg_user_year(agg_user_year_list)
Aggrigation_user_state = tabulate_agg_user_state(agg_user_state_list)
Map_trans_state = tabulate_map_trans_state(map_trans_state_list)
Map_user_state = tabulate_map_user_state(map_user_state_list)
Top_trans_year = tabulate_top_trans_year(top_trans_year_list)
Top_trans_state = tabulate_top_trans_state(top_trans_state_list)
Top_user_year = tabulate_top_user_year(top_user_year_list)
Top_user_state = tabulate_top_user_state(top_user_state_list)

import sqlite3
conn = sqlite3.connect("Tamil.db")
cur = conn.cursor()

#TABLE 1:
cur.execute('''CREATE TABLE IF NOT EXISTS AGGRIGATION_TRANSACTION_YEARWISE(
            YEAR INT,
            QUADRANT TINYINT,
            CATEGORY VARCHAR(30),
            TOTAL_COUNT INT,
            TOTAL_AMOUNT FLOAT
)''')

for (i, j) in Aggrigation_trans_year.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO AGGRIGATION_TRANSACTION_YEARWISE VALUES{dp}''')

#TABLE 2:
cur.execute('''
        CREATE TABLE IF NOT EXISTS AGGREGATION_TRANSACTION_STATEWISE(
          STATE_NAME VARCHAR(35),
          YEAR INT,
          QUADRANT TINYINT,
          PEER_TO_PEER_PAYMENT_COUNT INT,
          PEER_TO_PEER_PAYMENT_VALUE FLOAT,
          MERCHANT_PAYMENTS_COUNT INT,
          MERCHANT_PAYMENTS_VALUE FLOAT,
          RECHARGE_BILL_PAYMENTS_COUNT INT,
          RECHARGE_BILL_PAYMENTS_VALUE FLOAT,
          FINANCIAL_SERVICES_COUNT INT,
          FINANCIAL_SERVICES_VALUE FLOAT,
          OTHERS_COUNT INT,
          OTHERS_VALUE FLOAT,
          TOTAL_COUNT INT,
          TOTAL_VALUE FLOAT
        )
''')
#since the numeric column data is filled with nan we are replacing nan with 0
Aggrigation_trans_state.isnull().sum()
Aggrigation_trans_state.fillna(0, axis = 1, inplace = True)

#Overall aggrigated count and amount of the states over Years
count_cols = []
val_cols = []
for i in Aggrigation_trans_state.columns:
  if i.endswith("Count"):
    count_cols.append(i)
  elif i.endswith("Value"):
    val_cols.append(i)

count_cols.insert(0, "State Name")
count_df = Aggrigation_trans_state[count_cols]
count_col_new = count_df.sum(axis = 1, skipna = True, numeric_only=True).tolist()

val_cols.insert(0, "State Name")
val_df = Aggrigation_trans_state[val_cols]
val_col_new = val_df.sum(axis = 1, skipna = True, numeric_only=True).tolist()

#Adding the new columns to the datafram 
Aggrigation_trans_state["Total Count"] = count_col_new
Aggrigation_trans_state["Total Amount"] = val_col_new

for (i, j) in Aggrigation_trans_state.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO AGGREGATION_TRANSACTION_STATEWISE VALUES{dp}''')

#TABLE 3:
cur.execute('''
        CREATE TABLE IF NOT EXISTS AGGREGATED_USER_YEARWISE(
          YEAR INT,
          QUADRANT INT,
          REGISTERED_USERS INT,
          APP_OPENS INT,
          XIAOMI_USERS INT,
          SAMSUNG_USERS INT,
          VIVO_USERS INT,
          OPPO_USERS INT,
          REALME_USERS INT,
          APPLE_USERS INT,
          MOTOROLA_USERS INT,
          ONEPLUS_USERS INT,
          HUAWEI_USERS INT,
          TECHNO_USERS INT,
          OTHERS_USERS INT,
          LENOVO_USERS INT
        )
''')
#since the data was updated 6 month once these datas are left null for long days so it is filled with 0
Aggrigation_user_year_nonull = Aggrigation_user_year.iloc[:-3].copy()
Aggrigation_user_year_nonull.fillna(0, inplace = True)

#columns containing all values are updated into database seperately
for (i, j) in Aggrigation_user_year_nonull.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO AGGREGATED_USER_YEARWISE VALUES{dp}''')

#since some records was not uploaded the null columns are uploades seperately
for (i, j) in Aggrigation_user_year.iloc[-3:, :4].iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO AGGREGATED_USER_YEARWISE(YEAR, QUADRANT, REGISTERED_USERS, APP_OPENS) VALUES{dp}''')

#TABLE : 4
cur.execute('''
        CREATE TABLE IF NOT EXISTS AGGREGATED_USER_STATEWISE(
          STATE_NAME VARCHAR(35),
          YEAR INT,
          QUADRANT INT,
          REGISTERED_USERS INT,
          APP_OPENS INT,
          XIAOMI_USERS INT,
          VIVO_USERS INT,
          OPPO_USERS INT,
          SAMSUNG_USERS INT, 
          REALME_USERS INT,
          APPLE_USERS INT,
          HUAWEI_USERS INT,
          GIONEE_USERS INT,
          ONEPLUS_USERS INT,
          TECHNO_USERS INT,
          OTHERS_USERS INT,
          ASUS_USERS INT,
          MOTOROLA_USERS INT,
          LENOVO_USERS INT,
          COOLPAD_USERS INT,
          MICROMAX_USERS,
          HMD_GLOBAL_USERS,
          INFINIX_USERS,
          LAVA_USERS,
          LYF_USERS INT         
        )
''')
Aggrigation_user_state_nonull = Aggrigation_user_state.iloc[:612, :].copy()
Aggrigation_user_state_nonull.fillna(0, inplace = True)
Aggrigation_user_state_null = Aggrigation_user_state.iloc[612:, :5].copy()
#columns containing all values are updated into database seperately
for (i, j) in Aggrigation_user_state_nonull.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO AGGREGATED_USER_STATEWISE VALUES{dp}''')

#since some records was not uploaded the null columns are uploades seperately
for (i, j) in Aggrigation_user_state_null.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO AGGREGATED_USER_STATEWISE(STATE_NAME, YEAR, QUADRANT, REGISTERED_USERS, APP_OPENS) VALUES{dp}''')

#TABLE : 5
cur.execute('''
            CREATE TABLE IF NOT EXISTS MAP_TRANSACTION_STATEWISE(
              YEAR INT,
              QUADRANT TINYINT,
              STATE_NAME VARCHAR(35),
              DISTRICT_NAME VARCHAR(50),
              TRANSACTION_COUNT INT,
              TRANSACTION_AMOUNT FLOAT
            )
''')
for (i, j) in Map_trans_state.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO MAP_TRANSACTION_STATEWISE VALUES{dp}''')

#TABLE : 6
cur.execute('''
            CREATE TABLE IF NOT EXISTS MAP_USER_STATEWISE(
              YEAR INT,
              QUADRANT TINYINT,
              STATE_NAME VARCHAR(35),
              DISTRICT_NAME VARCHAR(50),
              REGISTERED_USERS INT,
              APP_OPENS INT
            )
''')
for (i, j) in Map_user_state.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO MAP_USER_STATEWISE VALUES{dp}''')

#TABLE : 7
cur.execute('''
            CREATE TABLE IF NOT EXISTS TOP_TRANSACTION_YEARWISE(
              YEAR INT,
              QUADRANT TINYINT,
              TOP_CITY_NAME VARCHAR(30),
              CITY_TRANSACTION_COUNT INT,
              CITY_TRANSACTION_AMOUNT FLOAT,
              TOP_PINCODE VARCHAR(6),
              PINCODE_TRANSACTION_COUNT INT,
              PINCODE_TRANSACTION_AMOUNT FLOAT,
              TOP_STATE_NAME VARCHAR(35),
              STATE_TRANSACTION_COUNT INT,
              STATE_TRANSACTION_AMOUNT FLOAT
            )
''')
for (i, j) in Top_trans_year.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO TOP_TRANSACTION_YEARWISE VALUES{dp}''')

#TABLE : 8
cur.execute('''
            CREATE TABLE IF NOT EXISTS TOP_TRANSACTION_STATEWISE(
              STATE_NAME VARCHAR(35),
              YEAR INT,
              QUADRANT TINYINT,
              TOP_DISTRICT_NAME VARCHAR(50),
              DISTRICT_TRANSACTION_COUNT INT,
              DISTRICT_TRANSACTION_AMOUNT FLOAT,
              TOP_PINCODE VARCHAR(6),
              PINCODE_TRANSACTION_COUNT INT,
              PINCODE_TRANSACTION_AMOUNT FLOAT
            )
''')
#Since some small states were present in the table some have less than 10 districts and have more pincodes. the district columns were left as null for those states. So the null valued columns are seperated and inserted seperately
Null_valued_index_pin = Top_trans_state[Top_trans_state["Top Pincode"].isna()].index
Top_trans_state.drop(Null_valued_index_pin, inplace = True)
Top_trans_state_nonull = Top_trans_state.copy() 
Top_trans_state_nonull.dropna(axis = 0, inplace = True)

for (i, j) in Top_trans_state_nonull.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO TOP_TRANSACTION_STATEWISE VALUES{dp}''')

for (i, j) in Top_trans_state[Top_trans_state["Top District Name"].isna()].iloc[: , [0, 1, 2, 6, 7, 8]].iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO TOP_TRANSACTION_STATEWISE(STATE_NAME, YEAR, QUADRANT,TOP_PINCODE, PINCODE_TRANSACTION_COUNT, PINCODE_TRANSACTION_AMOUNT) VALUES{dp}''')

#TABLE : 9
cur.execute('''
            CREATE TABLE IF NOT EXISTS TOP_USER_YEARWISE(
              YEAR INT,
              QUADRANT TINYINT,
              TOP_DISTRICT_NAME VARCHAR(30),
              DISTRICT_REGISTERED_USERS_COUNT INT,
              TOP_PINCODE VARCHAR(6),
              PINCODE_REGISTERED_USERS_COUNT INT
            )
''')
for (i, j) in Top_user_year.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO TOP_USER_YEARWISE VALUES{dp}''')

#TABLE : 10
cur.execute('''
              CREATE TABLE IF NOT EXISTS TOP_USER_STATEWISE(
                STATE_NAME VARCHAR(35),
                YEAR INT,
                QUADRANT TINYINT,
                TOP_DISTRICT_NAME VARCHAR(50),
                DISTRICT_REGISTERED_USERS_COUNT INT,
                TOP_PINCODE VARCHAR(6),
                PINCODE_REGISTERED_USERS_COUNT INT
              )
''')
#Since some small states were present in the table some have less than 10 districts and have more pincodes. the district columns were left as null for those states. So the null valued columns are seperated and inserted seperately
Top_user_state_nonull = Top_user_state.copy() 
Top_user_state_nonull.dropna(axis = 0, inplace = True)
for (i, j) in Top_user_state_nonull.iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO TOP_USER_STATEWISE VALUES{dp}''')

for (i, j) in Top_user_state[Top_user_state["Top District Name"].isna()].iloc[:, [0, 1,2, 5, 6]].iterrows():
  dp = tuple(j)
  cur.execute(f'''INSERT INTO TOP_USER_STATEWISE(STATE_NAME, YEAR, QUADRANT,TOP_PINCODE, PINCODE_REGISTERED_USERS_COUNT) VALUES{dp}''')





#fetching the inserted data TABLE : 1
fetch_data1 = cur.execute("select * from AGGRIGATION_TRANSACTION_YEARWISE").fetchall()

#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(AGGRIGATION_TRANSACTION_YEARWISE)"):
  cols.append(i[1])
new_df1 = pd.DataFrame(fetch_data1, columns = cols)

#fetching the inserted data TABLE : 2
fetch_data2 = cur.execute("select * from AGGREGATION_TRANSACTION_STATEWISE")
fetch_data2 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(AGGREGATION_TRANSACTION_STATEWISE)"):
  cols.append(i[1])
new_df2 = pd.DataFrame(fetch_data2, columns = cols)

#fetching the inserted data TABLE : 3
fetch_data3 = cur.execute("select * from AGGREGATED_USER_YEARWISE")
fetch_data3 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(AGGREGATED_USER_YEARWISE)"):
  cols.append(i[1])
new_df3 = pd.DataFrame(fetch_data3, columns = cols)

#fetching the inserted data TABLE : 4
fetch_data4 = cur.execute("select * from AGGREGATED_USER_STATEWISE")
fetch_data4 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(AGGREGATED_USER_STATEWISE)"):
  cols.append(i[1])
new_df4 = pd.DataFrame(fetch_data4, columns = cols)

#fetching the inserted data TABLE : 5
fetch_data5 = cur.execute("select * from MAP_TRANSACTION_STATEWISE")
fetch_data5 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(MAP_TRANSACTION_STATEWISE)"):
  cols.append(i[1])
new_df5 = pd.DataFrame(fetch_data5, columns = cols)

#fetching the inserted data TABLE : 6
fetch_data6 = cur.execute("select * from MAP_USER_STATEWISE")
fetch_data6 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(MAP_USER_STATEWISE)"):
  cols.append(i[1])
new_df6 = pd.DataFrame(fetch_data6, columns = cols)

#TABLE : 7
#fetching the inserted data
fetch_data7 = cur.execute("select * from TOP_TRANSACTION_YEARWISE")
fetch_data7 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(TOP_TRANSACTION_YEARWISE)"):
  cols.append(i[1])
new_df7 = pd.DataFrame(fetch_data7, columns = cols)

#fetching the inserted data TABLE : 8
fetch_data8 = cur.execute("select * from TOP_TRANSACTION_STATEWISE")
fetch_data8 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(TOP_TRANSACTION_STATEWISE)"):
  cols.append(i[1])
new_df8 = pd.DataFrame(fetch_data8, columns = cols)

#fetching the inserted data TABLE : 9
fetch_data9 = cur.execute("select * from TOP_USER_YEARWISE")
fetch_data9 = cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(TOP_USER_YEARWISE)"):
  cols.append(i[1])
new_df9 = pd.DataFrame(fetch_data9, columns = cols)

#fetching the inserted data TABLE : 10
fetch_data10 = cur.execute("select * from TOP_USER_STATEWISE")
fetch_data10= cur.fetchall()
#fetching the column names of the data
cols = []
for i in cur.execute("PRAGMA table_info(TOP_USER_STATEWISE)"):
  cols.append(i[1])
new_df10 = pd.DataFrame(fetch_data10, columns = cols)



Overwriting etl.py


In [ ]:
%%writefile visuvalization.py
import pandas as pd


def year_agg_trans(df, yr, qd):
  year = yr
  quadrant = qd
  df11 = df

  if year == "All Years":
    df11 = df11
  elif year != "All Years":
    df11 = df11[df11["YEAR"] == year]
  
  if quadrant == "All Quadrants":
    df11 = df11
  elif quadrant != "All Quadrants":
    df11 = df11[df11["QUADRANT"] == quadrant]



  from plotly.subplots import make_subplots
  import plotly.graph_objects as go

  fig = make_subplots(rows=3, cols=2, 
                      specs=[[{"type": "bar"}, {"type": "bar"}],
                            [{"type": "pie"}, {"type": "pie"}],
                            [{"type": "bar"}, {"type": "bar"}]],
                      subplot_titles=("Year vs Transaction count","Year vs Transaction amount", "Catogery vs Transaction Count", "Catogery vs Transaction amount","Quadrant vs Transaction Count", "Quadrant vs Transaction Amount" ))

  fig.add_trace(
      go.Bar(y = df11.groupby("YEAR").sum()["TOTAL_COUNT"],x = df11.groupby("YEAR").sum().index),
      row=1, col=1
  )

  fig.add_trace(
      go.Bar(y = df11.groupby("YEAR").sum()["TOTAL_AMOUNT"],x = df11.groupby("YEAR").sum().index),
      row=1, col=2
  )

  fig.add_trace(
      go.Pie(values = df11.groupby("CATEGORY").sum()["TOTAL_COUNT"], labels = df11.groupby("CATEGORY").sum().index),
      row=2, col=1
  )

  fig.add_trace(
      go.Pie(values = df11.groupby("CATEGORY").sum()["TOTAL_AMOUNT"], labels = df11.groupby("CATEGORY").sum().index),
      row=2, col=2
  )

  fig.add_trace(
      go.Bar(y = df11.groupby("QUADRANT").sum()["TOTAL_COUNT"],x = df11.groupby("QUADRANT").sum().index),
      row=3, col=1
  )

  fig.add_trace(
      go.Bar(y = df11.groupby("QUADRANT").sum()["TOTAL_AMOUNT"],x = df11.groupby("QUADRANT").sum().index),
      row=3, col=2
  )

  fig.update_layout(height = 1100, width=1000, title_text="Aggrigation of yearly transaction", showlegend=False)
  return fig

def year_agg_user(df, yr, qd):
  year = yr
  quadrant = qd
  df13 = df
  #new_df3
  if year == "All Years":
    df13 = df13
  elif year != "All Years":
    df13 = df13[df13["YEAR"] == year]

  if quadrant == "All Quadrants":
    df13 = df13
  elif quadrant != "All Quadrants":
    df13 = df13[df13["QUADRANT"] == quadrant]

  #Since Year 2022 Q -> 2, 3, 4 data was not updated we are considering the data of 2018 to 2022 Q1 for device ratio
  st_user_device_year = df13.iloc[:-3, :]

  from plotly.subplots import make_subplots
  import plotly.graph_objects as go

  fig = make_subplots(rows=3, cols=2, 
                      specs=[[{"type": "bar"}, {"type": "bar"}],
                             [{"type": "bar"}, {"type": "bar"}],
                             [{"type": "pie"}, {"type": "pie"}]],
                      subplot_titles=("Year vs Registered Users","Year vs App Opens", "Quadrant vs Registered Users", "Quadrant vs App opens","Users vs Device used" ))

  fig.add_trace(
      go.Bar(y = df13.groupby("YEAR").sum()["REGISTERED_USERS"],x = df13.groupby("YEAR").sum().index),
      row=1, col=1
  )

  fig.add_trace(
      go.Bar(y = df13.groupby("YEAR").sum()["APP_OPENS"],x = df13.groupby("YEAR").sum().index),
      row=1, col=2
  )


  fig.add_trace(
      go.Bar(x =df13.groupby("QUADRANT").sum().iloc[:, 1:3].index, y = df13.groupby("QUADRANT").sum().iloc[:, 1:3]["REGISTERED_USERS"]),
      row=2, col=1
  )

  fig.add_trace(
      go.Bar(x =df13.groupby("QUADRANT").sum().iloc[:, 1:3].index, y = df13.groupby("QUADRANT").sum().iloc[:, 1:3]["APP_OPENS"]),
      row=2, col=2
  )

  fig.add_trace(
      go.Pie(values = st_user_device_year.groupby("YEAR").sum().sum().tolist()[3:], labels = st_user_device_year.groupby("YEAR").sum().columns[3:]),
      row=3, col=1
  )



  fig.update_layout(height = 1100, width=1000, title_text="Aggrigation of yearly users", showlegend=False)
  return fig

def state_agg_trans(df1,df2,st, yr, qd):
  import pandas as pd
  import plotly.express as px

  year = yr
  quadrant = qd
  state = st
  df12 = df1
  df14 = df2
  #new_df2
  if state == "All States":
    df12 = df12
    df14 = df14
  elif state != "All States":
    df12 = df12[df12["STATE_NAME"] == state]
    df14 = df14[df14["STATE_NAME"] == state]
  if year == "All Years":
    df12 = df12
    df14 = df14
  elif year != "All Years":
    df12 = df12[df12["YEAR"] == year]
    df14 = df14[df14["YEAR"] == year]
  if quadrant == "All Quadrants":
    df12 = df12
    df14 = df14
  elif quadrant != "All Quadrants":
    df12 = df12[df12["QUADRANT"] == quadrant]
    df14 = df14[df14["QUADRANT"] == quadrant]

  df1214 = df12.iloc[:, [0, 1, 2, -1, -2]].groupby("STATE_NAME").sum().iloc[:, 2:].merge(df14.iloc[:, :5].groupby("STATE_NAME").sum().iloc[:,2:], on = df12.iloc[:, [0, 1, 2, -1, -2]].groupby("STATE_NAME").sum().index, how = "inner")
  df1214.columns = ["STATE_NAME", "TOTAL_VALUE", "TOTAL_COUNT","REGISTERED_USERS","APP_OPENS"]

  fig = px.choropleth(
      df1214,
      geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
      featureidkey='properties.ST_NM',
      locations='STATE_NAME',
      color= "TOTAL_VALUE",
      color_continuous_scale='Reds',
      hover_name = "STATE_NAME",
      hover_data = df1214.columns,
  )

  fig.update_geos(fitbounds="locations", visible=False)

  fig.update_layout(
      title=dict(
              text="Aggregated Transaction count state wise",
              xanchor='center',
              x=0.5,
              yref='paper',
              yanchor='bottom',
              y=1,
              pad={'b':10}),
              margin={'r':0,'t':30,'l':0,'b':0},
              height=550,
              width=900
        )
  return fig



def state_map_trans(df, st, yr, qd):
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go

  #new_df5


  state = st
  year = yr
  quadrant = qd
  df15 = df
  if state == "All States":
    df15 = df15
  elif state != "All States":
    df15 = df15[df15["STATE_NAME"] == state]

  if year == "All Years":
    df15 = df15
  elif year != "All Years":
    df15 = df15[df15["YEAR"] == year]

  if quadrant == "All Quadrants":
    df15 = df15
  elif quadrant != "All Quadrants":
    df15 = df15[df15["QUADRANT"] == quadrant]


  fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=(f"Transaction Values of {state} by District wise of {year}",f"Transaction Count of {state} by District wise of {year}"))

  fig.add_trace(
      go.Bar(x =df15.groupby("DISTRICT_NAME").sum().sort_values("TRANSACTION_AMOUNT", ascending = False).index,
         y = df15.groupby("DISTRICT_NAME").sum().sort_values("TRANSACTION_AMOUNT", ascending = False)["TRANSACTION_AMOUNT"]),
      row=1, col=1
  )

  fig.add_trace(
      go.Bar(x = df15.groupby("DISTRICT_NAME").sum().sort_values("TRANSACTION_COUNT", ascending = False).index,
         y = df15.groupby("DISTRICT_NAME").sum().sort_values("TRANSACTION_COUNT", ascending = False)["TRANSACTION_COUNT"]),
      row=2, col=1
  )



  fig.update_yaxes(title_text = "Transaction Amount", row = 1, col = 1)
  fig.update_xaxes(title_text = "District Name", row = 1, col = 1)
  fig.update_yaxes(title_text = "Transaction Count", row = 2, col = 1)
  fig.update_xaxes(title_text = "District Name", row = 2, col = 1)


  fig.update_layout(height = 1100, width=1000, title_text=f"Districtwise aggrigation of {state}", showlegend=False)
  return fig


def state_map_user(df, st, yr, qd):
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go

  #new_df6
  state = st
  year = yr
  quadrant = qd
  df16  = df
  if state == "All States":
    df16 = df16
  elif state != "All States":
    df16 = df16[df16["STATE_NAME"] == state]

  if year == "All Years":
    df16 = df16
  elif year != "All Years":
    df16 = df16[df16["YEAR"] == year]

  if quadrant == "All Quadrants":
    df16 = df16
  elif quadrant != "All Quadrants":
    df16 = df16[df16["QUADRANT"] == quadrant]


  fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=(f"Registered users count of {state} by District wise of {year}",f"App opens count of {state} by District wise of {year}"))

  fig.add_trace(
      go.Bar(x =df16.groupby("DISTRICT_NAME").sum().sort_values("REGISTERED_USERS", ascending = False).index,
         y = df16.groupby("DISTRICT_NAME").sum().sort_values("REGISTERED_USERS", ascending = False)["REGISTERED_USERS"]),
      row=1, col=1
  )

  fig.add_trace(
      go.Bar(x = df16.groupby("DISTRICT_NAME").sum().sort_values("APP_OPENS", ascending = False).index,
         y = df16.groupby("DISTRICT_NAME").sum().sort_values("APP_OPENS", ascending = False)["APP_OPENS"]),
      row=2, col=1
  )



  fig.update_yaxes(title_text = "Registered Users count", row = 1, col = 1)
  fig.update_xaxes(title_text = "District Name", row = 1, col = 1)
  fig.update_yaxes(title_text = "App Opensss Count", row = 2, col = 1)
  fig.update_xaxes(title_text = "District Name", row = 2, col = 1)


  fig.update_layout(height = 1100, width=1000, title_text=f"Districtwise aggrigation of {state}", showlegend=False)
  return fig


def year_top_trans(df, yr, qd):
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  #new_df7
  df17 = df
  year = yr
  quadrant = qd
  if year == "All Years":
    df17 = df17
  elif year != "All Years":
    df17 = df17 = df17[df17["YEAR"] == year]

  if quadrant == "All Quadrants":
    df17 = df17
  elif quadrant != "All Quadrants":
    df17 = df17[df17["QUADRANT"] == quadrant]


  fig = make_subplots(rows=3, cols=2, 
                      specs=[[{"type": "bar"}, {"type": "bar"}],
                             [{"type": "pie"}, {"type": "pie"}],
                             [{"type": "bar"}, {"type": "bar"}]],
                      subplot_titles=(f"Top cities by Transaction count of {year}",f"Top cities by Transaction Amount of {year}", 
                                      f"Top Pincodes by Transaction count of {year}", f"Top Pincodes by Transaction Amount of {year}", 
                                      f"Top States by Transaction count of {year}", f"Top States by Transaction Amount of {year}"))

  fig.add_trace(
      go.Bar(x = df17.iloc[:, :5].groupby("TOP_CITY_NAME").sum().sort_values("CITY_TRANSACTION_COUNT", ascending = False).head(10).index,
             y = df17.iloc[:, :5].groupby("TOP_CITY_NAME").sum().sort_values("CITY_TRANSACTION_COUNT", ascending = False).head(10)["CITY_TRANSACTION_COUNT"]),
      row=1, col=1
  )

  fig.add_trace(
      go.Bar(x = df17.iloc[:, :5].groupby("TOP_CITY_NAME").sum().sort_values("CITY_TRANSACTION_AMOUNT", ascending = False).head(10).index,
             y = df17.iloc[:, :5].groupby("TOP_CITY_NAME").sum().sort_values("CITY_TRANSACTION_AMOUNT", ascending = False).head(10)["CITY_TRANSACTION_AMOUNT"]),
      row=1, col=2
  )

  fig.add_trace(
      go.Pie(values =df17.iloc[:, [0, 1, 5, 6, 7]].groupby("TOP_PINCODE").sum().sort_values("PINCODE_TRANSACTION_COUNT", ascending = False).head(10)["PINCODE_TRANSACTION_COUNT"],
             labels = df17.iloc[:, [0, 1, 5, 6, 7]].groupby("TOP_PINCODE").sum().sort_values("PINCODE_TRANSACTION_COUNT", ascending = False).head(10).index),
      row=2, col=1
  )

  fig.add_trace(
      go.Pie(values =df17.iloc[:, [0, 1, 5, 6, 7]].groupby("TOP_PINCODE").sum().sort_values("PINCODE_TRANSACTION_AMOUNT", ascending = False).head(10)["PINCODE_TRANSACTION_AMOUNT"],
             labels = df17.iloc[:, [0, 1, 5, 6, 7]].groupby("TOP_PINCODE").sum().sort_values("PINCODE_TRANSACTION_AMOUNT", ascending = False).head(10).index),
      row=2, col=2
  )

  fig.add_trace(
      go.Bar(y =df17.iloc[:, [0, 1, -3, -2, -1]].groupby("TOP_STATE_NAME").sum().sort_values("STATE_TRANSACTION_COUNT", ascending = False).head(10)["STATE_TRANSACTION_COUNT"],
             x = df17.iloc[:, [0, 1, -3, -2, -1]].groupby("TOP_STATE_NAME").sum().sort_values("STATE_TRANSACTION_COUNT", ascending = False).head(10).index),
      row=3, col=1
  )

  fig.add_trace(
      go.Bar(y =df17.iloc[:, [0, 1, -3, -2, -1]].groupby("TOP_STATE_NAME").sum().sort_values("STATE_TRANSACTION_AMOUNT", ascending = False).head(10)["STATE_TRANSACTION_AMOUNT"],
             x = df17.iloc[:, [0, 1, -3, -2, -1]].groupby("TOP_STATE_NAME").sum().sort_values("STATE_TRANSACTION_AMOUNT", ascending = False).head(10).index),
      row=3, col=2
  )

  #Updating X-axis label
  fig.update_xaxes(title_text="City Name", row=1, col=1)
  fig.update_xaxes(title_text="City Name", row=1, col=2)

  fig.update_xaxes(title_text="State", row=3, col=1)
  fig.update_xaxes(title_text="State", row=3, col=2)
  #Updating Y-axis label
  fig.update_yaxes(title_text="Transaction Count", row=1, col=1)
  fig.update_yaxes(title_text="Transaction Amount", row=1, col=2)

  fig.update_yaxes(title_text="Transaction Count", row=3, col=1)
  fig.update_yaxes(title_text="Transaction Amount", row=3, col=2)

  fig.update_layout(height = 1100, width=1000, title_text=f"Top Transactions by City, Pincode, State of {year}", showlegend=False)
  return fig


import geopy.geocoders
from geopy.geocoders import Nominatim
def loc(x):
  try:
    geolocator = Nominatim(user_agent='google', timeout=3)
    location = geolocator.geocode(x)
    return ",".join(location.raw["display_name"].split(",")[:-3])
  except AttributeError:
    return "Not Known"

def state_top_trans(df, st, yr, qd):
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go

  df18 = df
  state = st
  year = yr
  quadrant = qd
  if state == "All States":
    df18_district = df18
    df18_pin = df18
  elif state != "All States":
    df18_district = df18[df18["STATE_NAME"] == state]
    df18_pin = df18[df18["STATE_NAME"] == state]
  

  if year == "All Years":
    df18_district = df18_district
    df18_pin = df18_pin
  elif year != "All Years":
    df18_district = df18_district[df18_district["YEAR"] == year]
    df18_pin = df18_pin[df18_pin["YEAR"] == year]

  if quadrant == "All Quadrants":
    df18_district = df18_district
    df18_pin = df18_pin
  elif quadrant != "All Quadrants":
    df18_district = df18_district[df18_district["QUADRANT"] == quadrant]
    df18_pin = df18_pin[df18_pin["QUADRANT"] == quadrant]


  #Step for adding the loccaion of the pincode
  pin_loc = {}
  for i in df18_pin["TOP_PINCODE"].unique():
    pin_loc[i] = i+"-"+loc(i)
  df18_pin["PINCODE_WITH_LOCATION"] = df18_pin["TOP_PINCODE"].map(pin_loc)




  fig = make_subplots(rows=2, cols=2, 
                      specs=[[{"type": "pie"}, {"type": "pie"}],
                             [{"type": "pie"}, {"type": "pie"}]],
                      subplot_titles=(f"Top Districts of the {state} by Transaction Count of {year}",f"Top cities of {state} by Transaction Amount of {year}", f"Top Pincodes of {state} by Transaction count of {year}", f"Top Pincodes of {state} by Transaction Amount of {year}"))

  fig.add_trace(
      go.Pie(labels= df18_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_TRANSACTION_COUNT", ascending  = False).index,
         values = df18_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_TRANSACTION_COUNT", ascending  = False)["DISTRICT_TRANSACTION_COUNT"]),
      row=1, col=1
  )

  fig.add_trace(
      go.Pie(labels= df18_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_TRANSACTION_AMOUNT", ascending  = False).index,
         values = df18_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_TRANSACTION_AMOUNT", ascending  = False)["DISTRICT_TRANSACTION_AMOUNT"]),
      row=1, col=2
  )

  fig.add_trace(
      go.Pie(labels= df18_pin.groupby("PINCODE_WITH_LOCATION").sum().sort_values("PINCODE_TRANSACTION_COUNT", ascending  = False).head(10).index,
         values = df18_pin.groupby("PINCODE_WITH_LOCATION").sum().sort_values("PINCODE_TRANSACTION_COUNT", ascending  = False).head(10)["PINCODE_TRANSACTION_COUNT"]),
      row=2, col=1
  )

  fig.add_trace(
      go.Pie(labels= df18_pin.groupby("PINCODE_WITH_LOCATION").sum().sort_values("PINCODE_TRANSACTION_AMOUNT", ascending  = False).head(10).index,
         values = df18_pin.groupby("PINCODE_WITH_LOCATION").sum().sort_values("PINCODE_TRANSACTION_AMOUNT", ascending  = False).head(10)["PINCODE_TRANSACTION_AMOUNT"]),
      row=2, col=2
  )


  fig.update_layout(height = 1100, width=1000, title_text=f"Top Transactions by City, Pincode of {state} of {year}", showlegend=False)
  return fig



def year_top_user(df, yr, qd):
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  #new_df9
  df19 = df
  df19_district = df19.iloc[:, :4]
  df19_pin = df19.iloc[:, [0,1, 4, 5]]
  year = yr
  quadrant = qd

  #filter for Districts
  if year == "All Years":
    df19_district = df19_district
    df19_pin = df19_pin
  elif year != "All Years":
    df19_district = df19_district[df19_district["YEAR"] == year]
    df19_pin = df19_pin[df19_pin["YEAR"] == year]

  if quadrant == "All Quadrants":
    df19_district = df19_district
    df19_pin = df19_pin
  elif quadrant != "All Quadrants":
    df19_district = df19_district[df19_district["QUADRANT"] == quadrant]
    df19_pin = df19_pin[df19_pin["QUADRANT"] == quadrant]

  #Step for adding the loccaion of the pincode
  pin_loc = {}
  for i in df19_pin["TOP_PINCODE"].unique():
    pin_loc[i] = i+"-"+loc(i)
  df19_pin["PINCODE_WITH_LOCATION"] = df19_pin["TOP_PINCODE"].map(pin_loc)
  


  fig = make_subplots(rows=2, cols=1,
                      specs=[[{"type": "bar"}], [{"type": "pie"}]],
                      subplot_titles=(f"Top cities by Registered Users of {year}",f"Top Pincode by Registered Users of {year}"))

  fig.add_trace(
      go.Bar(x = df19_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_REGISTERED_USERS_COUNT", ascending = False).head(10).index,
             y = df19_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_REGISTERED_USERS_COUNT", ascending = False).head(10)["DISTRICT_REGISTERED_USERS_COUNT"]),
      row=1, col=1
  )

  fig.add_trace(
      go.Pie(labels = df19_pin.groupby("PINCODE_WITH_LOCATION").sum().sort_values("PINCODE_REGISTERED_USERS_COUNT", ascending = False).head(10).index,
             values = df19_pin.groupby("PINCODE_WITH_LOCATION").sum().sort_values("PINCODE_REGISTERED_USERS_COUNT", ascending = False).head(10)["PINCODE_REGISTERED_USERS_COUNT"]),
      row=2, col=1
  )


  #Updating X-axis label
  fig.update_xaxes(title_text="District Name", row=1, col=1)

  #Updating Y-axis label
  fig.update_yaxes(title_text="Registered users Count", row=1, col=1)

  fig.update_layout(height = 1100, width=1000, title_text=f"Top users by city and Pincode", showlegend=False)
  return fig


def state_top_user(df, st, yr, qd):
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go

  df110 = df
  df110_district = df110.iloc[:, 0:5]
  df110_pin = df110.iloc[:, [0, 1, 2, 5, 6]]
  state = st
  year = yr
  quadrant = qd
    
  if state == "All States":
    df110_district = df110_district
    df110_pin = df110_pin
  elif state != "All States":
    df110_district = df110_district[df110_district["STATE_NAME"] == state]
    df110_pin = df110_pin[df110_pin["STATE_NAME"] == state]

  if year == "All Years":
    df110_district = df110_district
    df110_pin = df110_pin
  elif year != "All Years":
    df110_district = df110_district[df110_district["YEAR"] == year]
    df110_pin = df110_pin[df110_pin["YEAR"] == year]
  
  if quadrant == "All Quadrants":
    df110_district = df110_district
    df110_pin = df110_pin
  elif quadrant != "All Quadrants":
    df110_district = df110_district[df110_district["QUADRANT"] == quadrant]
    df110_pin = df110_pin[df110_pin["QUADRANT"] == quadrant]



  #Step for adding the loccaion of the pincode
  df_filter_user_st_pin_reg = df110_pin.groupby("TOP_PINCODE").sum().sort_values("PINCODE_REGISTERED_USERS_COUNT", ascending  = False).head(10)
  df_filter_user_st_pin_reg["TOP_PINCODE"] = df_filter_user_st_pin_reg.index
  df_filter_user_st_pin_reg["PINCODE_LOCATION"] = df_filter_user_st_pin_reg["TOP_PINCODE"].apply(lambda x:loc(x))
  pin_with_loc = {}
  for i, j in zip(df_filter_user_st_pin_reg["PINCODE_LOCATION"].tolist(), df_filter_user_st_pin_reg["TOP_PINCODE"].tolist()):
    pin_with_loc[j] = j+"-"+i
  df_filter_user_st_pin_reg["PINCODE_WITH_LOCATION"] = df_filter_user_st_pin_reg["TOP_PINCODE"].map(pin_with_loc)

  fig = make_subplots(rows=2, cols=1, 
                      specs=[[{"type": "bar"}], [{"type": "pie"}]],
                      subplot_titles=(f"Top Districts of {state} by Registered Users of {year}", f"Top Pincodes of {state} by Registered Users of {year}"))

  fig.add_trace(
      go.Bar(x= df110_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_REGISTERED_USERS_COUNT", ascending  = False).head(10).index,
         y = df110_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_REGISTERED_USERS_COUNT", ascending  = False).head(10)["DISTRICT_REGISTERED_USERS_COUNT"]),
      row=1, col=1
  )

  fig.add_trace(
      go.Pie(labels= df_filter_user_st_pin_reg["PINCODE_WITH_LOCATION"],
         values = df_filter_user_st_pin_reg["PINCODE_REGISTERED_USERS_COUNT"]),
      row=2, col=1
  )

  fig.update_layout(height = 1100, width=1000, title_text=f"Top Transactions by City, Pincode of {state} of {year}", showlegend=False)
  return fig

In [4]:
def state_top_user(df, st, yr, qd):
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go

  df110 = df
  df110_district = df110.iloc[:, 0:5]
  df110_pin = df110.iloc[:, [0, 1, 2, 5, 6]]
  state = st
  year = yr
  quadrant = qd
    
  if state == "All States":
    df110_district = df110_district
    df110_pin = df110_pin
  elif state != "All States":
    df110_district = df110_district[df110_district["STATE_NAME"] == state]
    df110_pin = df110_pin[df110_pin["STATE_NAME"] == state]

  if year == "All Years":
    df110_district = df110_district
    df110_pin = df110_pin
  elif year != "All Years":
    df110_district = df110_district[df110_district["YEAR"] == year]
    df110_pin = df110_pin[df110_pin["YEAR"] == year]
  
  if quadrant == "All Quadrants":
    df110_district = df110_district
    df110_pin = df110_pin
  elif quadrant != "All Quadrants":
    df110_district = df110_district[df110_district["QUADRANT"] == quadrant]
    df110_pin = df110_pin[df110_pin["QUADRANT"] == quadrant]



  #Step for adding the loccaion of the pincode
  df_filter_user_st_pin_reg = df110_pin.groupby("TOP_PINCODE").sum().sort_values("PINCODE_REGISTERED_USERS_COUNT", ascending  = False).head(10)
  df_filter_user_st_pin_reg["TOP_PINCODE"] = df_filter_user_st_pin_reg.index
  df_filter_user_st_pin_reg["PINCODE_LOCATION"] = df_filter_user_st_pin_reg["TOP_PINCODE"].apply(lambda x:loc(x))
  pin_with_loc = {}
  for i, j in zip(df_filter_user_st_pin_reg["PINCODE_LOCATION"].tolist(), df_filter_user_st_pin_reg["TOP_PINCODE"].tolist()):
    pin_with_loc[j] = j+"-"+i
  df_filter_user_st_pin_reg["PINCODE_WITH_LOCATION"] = df_filter_user_st_pin_reg["TOP_PINCODE"].map(pin_with_loc)

  fig = make_subplots(rows=2, cols=1, 
                      specs=[[{"type": "bar"}], [{"type": "pie"}]],
                      subplot_titles=(f"Top Districts of {state} by Registered Users of {year}", f"Top Pincodes of {state} by Registered Users of {year}"))

  fig.add_trace(
      go.Bar(x= df110_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_REGISTERED_USERS_COUNT", ascending  = False).head(10).index,
         y = df110_district.groupby("TOP_DISTRICT_NAME").sum().sort_values("DISTRICT_REGISTERED_USERS_COUNT", ascending  = False).head(10)["DISTRICT_REGISTERED_USERS_COUNT"]),
      row=1, col=1
  )

  fig.add_trace(
      go.Pie(labels= df_filter_user_st_pin_reg["PINCODE_WITH_LOCATION"],
         values = df_filter_user_st_pin_reg["PINCODE_REGISTERED_USERS_COUNT"]),
      row=2, col=1
  )

  fig.update_layout(height = 1100, width=1000, title_text=f"Top Transactions by City, Pincode of {state} of {year}", showlegend=False)
  return fig


In [ ]:
!pip install streamlit -q
!pip install pyngrok

In [ ]:
%%writefile mock_app.py
import pandas as pd
import streamlit as st
from etl import new_df1, new_df2, new_df3, new_df4, new_df5, new_df6, new_df7, new_df8, new_df9, new_df10
from visuvalization import *
def main():
  st.markdown("<h1 style='text-align: center; color: red;'>PHONEPE PULSE DATA VISUVALIZATION</h1>", unsafe_allow_html=True)
  
  st.markdown("<h2 style='text-align: center; color: red;'>YEARWISE VISUVALIZATION</h2>", unsafe_allow_html=True)
  col1, col2, col3 = st.columns(3)
  with col1:
    yr = st.selectbox("Select Year", ["All Years", 2018, 2019, 2020, 2021, 2022], key = "yearwise_year")
  with col2:
    qd = st.selectbox("Select Quadrant", ["All Quadrants", 1, 2, 3, 4], key = "yearwise_quadrant")
  with col3:
    type1 = st.selectbox("Select type", ["Aggrigated values", "Top Values"])
  type2 = st.radio("Select data", ["Transaction data", "Users data"])
  if type1 == "Aggrigated values" and type2 == "Transaction data":
    visual1 = year_agg_trans(new_df1, yr, qd)
  elif type1 == "Aggrigated values" and type2 == "Users data":
    visual1 = year_agg_user(new_df3, yr, qd)
  elif type1 == "Top Values" and type2 == "Transaction data":
    visual1 = year_top_trans(new_df7, yr, qd)
  elif type1 == "Top Values" and type2 == "Users data":
    visual1 = year_top_user(new_df9, yr, qd)
  
  st.write(visual1)
  st.markdown("<h2 style='text-align: center; color: red;'>STATEWISE MAP VISUVALIZATION</h2>", unsafe_allow_html=True)
  col4, col5, col6 = st.columns(3)
  with col4:
    state_1 = st.selectbox("Select the state", ["All States",'Mizoram','Lakshadweep','Meghalaya','Chhattisgarh','Karnataka','Odisha','Kerala','Goa','Uttarakhand','Punjab','Andhra Pradesh','Tamil Nadu',
    'Arunachal Pradesh','Chandigarh','Madhya Pradesh','West Bengal', 'Assam','Uttar Pradesh','Gujarat','Manipur','Tripura','Puducherry','Bihar','Nagaland','Rajasthan','Delhi',
    'Maharashtra','Telangana','Himachal Pradesh','Jharkhand','Haryana','Ladakh','Sikkim'])
  with col5:
    yr2 = st.selectbox("Select Year", ["All Years", 2018, 2019, 2020, 2021, 2022], key = "map_year")
  with col6:
    qd2 = st.selectbox("Select Quadrant", ["All Quadrants", 1, 2, 3, 4], key = "map_quadrant")
  visual2 = state_agg_trans(new_df2, new_df4, state_1, yr2, qd2)
  st.write(visual2)

  st.markdown("<h2 style='text-align: center; color: red;'>STATEWISE VISUVALIZATION</h2>", unsafe_allow_html=True)
  col7, col8, col9, col10= st.columns(4)
  with col7:
    state_2 = st.selectbox("Select State", ["All States", 'Mizoram','Lakshadweep','Meghalaya','Chhattisgarh','Karnataka','Odisha','Kerala','Goa','Uttarakhand','Punjab','Andhra Pradesh','Tamil Nadu','Arunachal Pradesh',
    'Chandigarh','Madhya Pradesh','West Bengal','Assam','Uttar Pradesh','Gujarat','Manipur','Tripura','Puducherry','Bihar','Jammu Kashmir','Nagaland','Rajasthan','Delhi','Maharashtra','Telangana',
    'Andaman Nicobar Islands','Himachal Pradesh','Jharkhand','Haryana','Ladakh','Sikkim','Dadra Nagar Haveli Daman Diu'], key = "state_2")
  with col8:
    yr3 = st.selectbox("Select Year", ["All Years", 2018, 2019, 2020, 2021, 2022], key = "state_year")
  with col9:
    qd3 = st.selectbox("Select Quadrant", ["All Quadrants", 1, 2, 3, 4], key = "state_quadrant")
  with col10:
    type4 = st.selectbox("Select Type", ["Aggrigated values", "Top values"], key = "state_type4")
  type5 = st.radio("Select Data Type", ["Transaction Data", "User Data"], key = "state_type5")
  if type4 == "Aggrigated values" and type5 == "Transaction Data":
    visuval3 = state_map_trans(new_df5, state_2, yr3, qd3)
  elif type4 == "Aggrigated values" and type5 == "User Data":
    visuval3 = state_map_user(new_df6, state_2, yr3, qd3)
  elif type4 == "Top values" and type5 == "Transaction Data":
    visuval3 = state_top_trans(new_df8, state_2, yr3, qd3)
  elif type4 == "Top values" and type5 == "User Data":
    visuval3 = state_top_user(new_df10, state_2, yr3, qd3)

   
  st.write(visuval3)

  new_df1_csv = new_df1.to_csv()
  new_df2_csv = new_df2.to_csv()
  new_df3_csv = new_df3.to_csv()
  new_df4_csv = new_df4.to_csv()
  new_df5_csv = new_df5.to_csv()
  new_df6_csv = new_df6.to_csv()
  new_df7_csv = new_df7.to_csv()
  new_df8_csv = new_df8.to_csv()
  new_df9_csv = new_df9.to_csv()
  new_df10_csv = new_df10.to_csv()

  st.write("For reference data click the download butten ...")
  if st.button("Download Data"):
    st.download_button("Download Table 1 as CSV",data = new_df1_csv,file_name = "Aggrigation_transaction_yearwise.csv",mime='text/csv')
    st.download_button("Download Table 2 as CSV",data = new_df2_csv,file_name = "Aggrigation_transaction_statewise.csv",mime='text/csv')
    st.download_button("Download Table 3 as CSV",data = new_df3_csv,file_name = "Aggrigation_user_yearwise.csv",mime='text/csv')
    st.download_button("Download Table 4 as CSV",data = new_df4_csv,file_name = "Aggrigation_user_statewise.csv",mime='text/csv')
    st.download_button("Download Table 5 as CSV",data = new_df5_csv,file_name = "Map_transaction_statewise.csv",mime='text/csv')
    st.download_button("Download Table 6 as CSV",data = new_df6_csv,file_name = "Map_user_statewise.csv",mime='text/csv')
    st.download_button("Download Table 7 as CSV",data = new_df7_csv,file_name = "Top_transaction_yearwise.csv",mime='text/csv')
    st.download_button("Download Table 8 as CSV",data = new_df8_csv,file_name = "Top_transaction_statewise.csv",mime='text/csv')
    st.download_button("Download Table 9 as CSV",data = new_df9_csv,file_name = "Top_user_yearwise.csv",mime='text/csv')
    st.download_button("Download Table 10 as CSV",data = new_df10_csv,file_name = "Top_user_statewise.csv",mime='text/csv')



  st.markdown("<h6 style='text-align: center; color: red;'>CREATED BY TAMIZHARASAN GOVINDASAMY</h6>", unsafe_allow_html=True)




if __name__ == "__main__":
  main()

In [ ]:
!pip install pyngrok==4.1.1

In [ ]:
!ngrok authtoken 2Kjk1sNkgtHmhrvADxwoX7sWOtY_87unGjkb4XkX7h8bLv2fs

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!streamlit run /content/mock_app.py 